In [1]:
import pandas as pd
import numpy as np
import os

print(pd.__version__)
print(np.__version__)

1.3.3
1.20.3


In [2]:
#Load in vivo data
pcarv_vivo_res = pd.read_csv("./luciferase_data/pcarv_vivo_response.csv", index_col = 0).reset_index(drop=True)
ncarv_vivo_res = pd.read_csv("./luciferase_data/ncarv_vivo_response.csv", index_col = 0).reset_index(drop=True)
pment_vivo_res = pd.read_csv("./luciferase_data/pment_vivo_response.csv", index_col = 0).reset_index(drop=True)
nment_vivo_res = pd.read_csv("./luciferase_data/nment_vivo_response.csv", index_col = 0).reset_index(drop=True)
pocta_vivo_res = pd.read_csv("./luciferase_data/pocta_vivo_response.csv", index_col = 0).reset_index(drop=True)
nocta_vivo_res = pd.read_csv("./luciferase_data/nocta_vivo_response.csv", index_col = 0).reset_index(drop=True)
#Load in vitro data
carv_vitro_res = pd.read_csv("./luciferase_data/carv_vitro_res.csv", index_col = 0).reset_index(drop=True)
ment_vitro_res = pd.read_csv("./luciferase_data/ment_vitro_res.csv", index_col = 0).reset_index(drop=True)
octa_vitro_res = pd.read_csv("./luciferase_data/octa_vitro_res.csv", index_col = 0).reset_index(drop=True)
#Combine in vivo data
carv_vivo_res = pd.concat([pcarv_vivo_res, ncarv_vivo_res]).reset_index(drop=True)
ment_vivo_res = pd.concat([pment_vivo_res, nment_vivo_res]).reset_index(drop=True)
octa_vivo_res = pd.concat([pocta_vivo_res, nocta_vivo_res]).reset_index(drop=True)


In [3]:
carv_results = {}
carv_results['receptor'] = {}
carv_results['vivo_pref'] = {}
carv_results['vitro_ratio'] = {}
carv_results['vivo_ratio'] = {}

carv_results['receptor']
counter = 0
for olfr in carv_vitro_res['tested_OR'].unique():
    carv_results['receptor'][counter] = olfr
    counter += 1
    
for num in carv_results['receptor']:
    olfr = carv_results['receptor'][num]
    olfr_dat = carv_vitro_res.loc[carv_vitro_res['tested_OR'] == olfr,]
    plusH_dat = olfr_dat.loc[olfr_dat['specific_odorant'] == 'plusH',]
    minusD_dat = olfr_dat.loc[olfr_dat['specific_odorant'] == 'minusD',]
    plusH_dat_nonZero = plusH_dat.loc[plusH_dat['specific_conc'] != 0,]
    minusD_dat_nonZero = minusD_dat.loc[minusD_dat['specific_conc'] != 0,]
    plusH_dat_zero = plusH_dat.loc[plusH_dat['specific_conc'] == 0,]
    minusD_dat_zero = minusD_dat.loc[minusD_dat['specific_conc'] == 0,]
    plus_avg_zero = plusH_dat_zero['normLuc_val'].values.mean()
    minus_avg_zero = minusD_dat_zero['normLuc_val'].values.mean()
    plus_auc = (plusH_dat_nonZero['normLuc_val'] / plus_avg_zero).sum()
    minus_auc = (minusD_dat_nonZero['normLuc_val'] / minus_avg_zero).sum()
    plus_auc = np.log2(plus_auc)
    minus_auc = np.log2(minus_auc)
    pn_ratio = plus_auc-minus_auc
    #if pn_ratio < 1:
    #    pn_ratio = (pn_ratio**(-1))*-1
    vivo_pref = carv_vivo_res.loc[carv_vivo_res['symbol'] == olfr,'response'].values[0]
    carv_results['vivo_pref'][num] = vivo_pref
    carv_results['vitro_ratio'][num] = pn_ratio
    plus_logfc = carv_vivo_res.loc[carv_vivo_res['symbol'] == olfr,]
    plus_logfc = plus_logfc.loc[plus_logfc['odor'] == 'plus',]['logFC']
    plus_logfc = plus_logfc.values[0]
    minus_logfc = carv_vivo_res.loc[carv_vivo_res['symbol'] == olfr,]
    minus_logfc = minus_logfc.loc[minus_logfc['odor'] == 'minus',]['logFC']
    minus_logfc = minus_logfc.values[0]
    logfc_pn_ratio = plus_logfc-minus_logfc
    #if logfc_pn_ratio < 1:
    #    logfc_pn_ratio = (logfc_pn_ratio**(-1))*-1
    carv_results['vivo_ratio'][num] = logfc_pn_ratio

In [4]:
ment_results = {}
ment_results['receptor'] = {}
ment_results['vivo_pref'] = {}
ment_results['vitro_ratio'] = {}
ment_results['vivo_ratio'] = {}

ment_results['receptor']
counter = 0
for olfr in ment_vitro_res['tested_OR'].unique():
    ment_results['receptor'][counter] = olfr
    counter += 1
    
for num in ment_results['receptor']:
    olfr = ment_results['receptor'][num]
    olfr_dat = ment_vitro_res.loc[ment_vitro_res['tested_OR'] == olfr,]
    plusH_dat = olfr_dat.loc[olfr_dat['specific_odorant'] == 'plusH',]
    minusD_dat = olfr_dat.loc[olfr_dat['specific_odorant'] == 'minusD',]
    plusH_dat_nonZero = plusH_dat.loc[plusH_dat['specific_conc'] != 0,]
    minusD_dat_nonZero = minusD_dat.loc[minusD_dat['specific_conc'] != 0,]
    plusH_dat_zero = plusH_dat.loc[plusH_dat['specific_conc'] == 0,]
    minusD_dat_zero = minusD_dat.loc[minusD_dat['specific_conc'] == 0,]
    plus_avg_zero = plusH_dat_zero['normLuc_val'].values.mean()
    minus_avg_zero = minusD_dat_zero['normLuc_val'].values.mean()
    plus_auc = (plusH_dat_nonZero['normLuc_val'] / plus_avg_zero).sum()
    minus_auc = (minusD_dat_nonZero['normLuc_val'] / minus_avg_zero).sum()
    plus_auc = np.log2(plus_auc)
    minus_auc = np.log2(minus_auc)
    pn_ratio = plus_auc-minus_auc
    #if pn_ratio < 1:
    #    pn_ratio = (pn_ratio**(-1))*-1
    vivo_pref = ment_vivo_res.loc[ment_vivo_res['symbol'] == olfr,'response'].values[0]
    ment_results['vivo_pref'][num] = vivo_pref
    ment_results['vitro_ratio'][num] = pn_ratio
    plus_logfc = ment_vivo_res.loc[ment_vivo_res['symbol'] == olfr,]
    plus_logfc = plus_logfc.loc[plus_logfc['odor'] == 'plus',]['logFC']
    plus_logfc = plus_logfc.values[0]
    minus_logfc = ment_vivo_res.loc[ment_vivo_res['symbol'] == olfr,]
    minus_logfc = minus_logfc.loc[minus_logfc['odor'] == 'minus',]['logFC']
    minus_logfc = minus_logfc.values[0]
    logfc_pn_ratio = plus_logfc-minus_logfc
    #if logfc_pn_ratio < 1:
    #    logfc_pn_ratio = (logfc_pn_ratio**(-1))*-1
    ment_results['vivo_ratio'][num] = logfc_pn_ratio

In [5]:
octa_results = {}
octa_results['receptor'] = {}
octa_results['vivo_pref'] = {}
octa_results['vitro_ratio'] = {}
octa_results['vivo_ratio'] = {}

octa_results['receptor']
counter = 0
for olfr in octa_vitro_res['tested_OR'].unique():
    octa_results['receptor'][counter] = olfr
    counter += 1
    
for num in octa_results['receptor']:
    olfr = octa_results['receptor'][num]
    olfr_dat = octa_vitro_res.loc[octa_vitro_res['tested_OR'] == olfr,]
    plusH_dat = olfr_dat.loc[olfr_dat['specific_odorant'] == 'plusH',]
    minusD_dat = olfr_dat.loc[olfr_dat['specific_odorant'] == 'minusD',]
    plusH_dat_nonZero = plusH_dat.loc[plusH_dat['specific_conc'] != 0,]
    minusD_dat_nonZero = minusD_dat.loc[minusD_dat['specific_conc'] != 0,]
    plusH_dat_zero = plusH_dat.loc[plusH_dat['specific_conc'] == 0,]
    minusD_dat_zero = minusD_dat.loc[minusD_dat['specific_conc'] == 0,]
    plus_avg_zero = plusH_dat_zero['normLuc_val'].values.mean()
    minus_avg_zero = minusD_dat_zero['normLuc_val'].values.mean()
    plus_auc = (plusH_dat_nonZero['normLuc_val'] / plus_avg_zero).sum()
    minus_auc = (minusD_dat_nonZero['normLuc_val'] / minus_avg_zero).sum()
    plus_auc = np.log2(plus_auc)
    minus_auc = np.log2(minus_auc)
    pn_ratio = plus_auc-minus_auc
    #if pn_ratio < 1:
    #    pn_ratio = (pn_ratio**(-1))*-1
    vivo_pref = octa_vivo_res.loc[octa_vivo_res['symbol'] == olfr,'response'].values[0]
    octa_results['vivo_pref'][num] = vivo_pref
    octa_results['vitro_ratio'][num] = pn_ratio
    plus_logfc = octa_vivo_res.loc[octa_vivo_res['symbol'] == olfr,]
    plus_logfc = plus_logfc.loc[plus_logfc['odor'] == 'plus',]['logFC']
    plus_logfc = plus_logfc.values[0]
    minus_logfc = octa_vivo_res.loc[octa_vivo_res['symbol'] == olfr,]
    minus_logfc = minus_logfc.loc[minus_logfc['odor'] == 'minus',]['logFC']
    minus_logfc = minus_logfc.values[0]
    logfc_pn_ratio = plus_logfc-minus_logfc
    #if logfc_pn_ratio < 1:
    #    logfc_pn_ratio = (logfc_pn_ratio**(-1))*-1
    octa_results['vivo_ratio'][num] = logfc_pn_ratio

In [6]:
carv_results = pd.DataFrame.from_dict(carv_results)
ment_results = pd.DataFrame.from_dict(ment_results)
octa_results = pd.DataFrame.from_dict(octa_results)
#carv_results.to_csv("./luciferase_data/carv_results.csv")
#ment_results.to_csv("./luciferase_data/ment_results.csv")
#octa_results.to_csv("./luciferase_data/octa_results.csv")